<a href="https://colab.research.google.com/github/erikitos/Python/blob/main/Flappy_Bird_beta_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pygame, random        # pygame: base del juego (ventana, imágenes, colisiones)
                             # random: genera valores aleatorios (alturas de tubos, etc.)

import sys                   # Permite interactuar con el sistema operativo (sys.exit())

from pygame.locals import *  # Importa constantes de Pygame (QUIT, KEYDOWN, K_SPACE...)
                             # Hace el código más limpio al manejar eventos.

import math                  # Funciones matemáticas: sin, cos, sqrt, radians...

import cv2                   # OpenCV: se usa si controlas el juego con cámara o gestos.

import mediapipe as mp       # Mediapipe: detección de manos, rostro o cuerpo (control por gestos).

SCREEN_WIDTH=500             # Define el tamaño de la ventana del juego:Todo lo que dibujes (pájaro, tubos, fondo) se adapta a este espacio.
SCREEN_HEIGHT=600            # Si usas cámara (cv2), también puedes ajustar el tamaño del frame para que coincida.
SPEED=12                     #SPEED: qué tan rápido sube el pájaro cuando salta (por ejemplo, al detectar un gesto con la mano).

GRAVITY=2.5                  #GRAVITY: qué tan rápido cae el pájaro. Si lo haces muy alto, el juego se vuelve difícil
GAME_SPEED=15                #GAME_SPEED: qué tan rápido se mueven los tubos hacia la izquierda. Afecta la dificultad general.
GROUND_HEIGHT=100            #GROUND_HEIGHT: altura del suelo. El pájaro pierde si lo toca.
PIPE_WIDTH=65                #PIPE_WIDTH: grosor de los tubos.


PIPE_HEIGHT=475              #PIPE_HEIGHT: altura máxima del tubo (el otro se calcula con el PIPE_GAP)
PIPE_GAP=225                  #PIPE_GAP: espacio entre los tubos superior e inferior. Si lo haces más pequeño, el juego se vuelve más difícil

pygame.init()  # Inicializa todos los módulos de Pygame. Es obligatorio antes de usar cualquier función de la librería.

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
# Crea la ventana del juego con las dimensiones definidas previamente.
# Aquí es donde se dibujará todo: fondo, pájaro, tubos, suelo, etc.

pygame.display.set_caption('Flappy Bird')
# Asigna un nombre a la ventana del juego. Aparece en la barra superior de la ventana.

BLUE = (77, 192, 202, 255)
# Define un color en formato RGBA (Rojo, Verde, Azul, Alfa).
# Este tono azul fue elegido para que combine con el fondo del cielo.
# El valor 255 de alfa indica que es completamente opaco.

WHITE = (255, 255, 255)
# Color blanco puro en formato RGB. Se usa para dibujar el texto del score o mensajes en pantalla.

background = pygame.image.load('images/fondoCielo.png')
# Carga la imagen del fondo desde la carpeta 'images'. Esta imagen representa el cielo.

background = pygame.transform.scale(background, (SCREEN_WIDTH, SCREEN_HEIGHT))
# Redimensiona la imagen del fondo para que ocupe toda la pantalla del juego.

bird = pygame.image.load('images/pajaroTieso.png')
# Carga la imagen del pájaro. En este caso, parece ser una versión estática del personaje.

bird = pygame.transform.scale(bird, (34, 24))
# Ajusta el tamaño del pájaro para que se vea bien en pantalla.
# Este tamaño influye en las colisiones y en cómo se mueve visualmente.

pipe_img = pygame.image.load('images/tuberia.png')
# Carga la imagen de la tubería (obstáculo). Es la parte inferior por defecto.

pipe_img = pygame.transform.scale(pipe_img, (PIPE_WIDTH, PIPE_HEIGHT))
# Redimensiona la tubería para que tenga el ancho y alto definidos en tus parámetros.

pipe_inverted_img = pygame.transform.flip(pipe_img, False, True)
# Crea una versión invertida verticalmente de la tubería.
# Se usa para el tubo superior, simulando que viene desde arriba.

grounds = pygame.image.load('images/base.png')
# Carga la imagen del suelo. Es la parte inferior donde el pájaro puede chocar.

grounds = pygame.transform.scale(grounds, (SCREEN_WIDTH, GROUND_HEIGHT))
# Ajusta el suelo para que tenga el ancho completo de la pantalla y la altura definida.


#rect[0] posicion de X, rect[1] posicion en y, rect[2] ancho y el 3 para altura
class Bird(pygame.sprite.Sprite):
       def __init__(self):
        pygame.sprite.Sprite.__init__(self)  # Inicializa el sprite base.

        self.image = pygame.Surface((34, 24), pygame.SRCALPHA)
        # Crea una superficie transparente del tamaño del pájaro.
        # SRCALPHA permite manejar transparencia (canal alfa).

        self.image.blit(bird, (0, 0))
        # Dibuja la imagen del pájaro sobre la superficie creada.

        self.speed = 0
        # Velocidad vertical del pájaro. Se usa para simular gravedad y salto.

        self.rect = self.image.get_rect()
        # Obtiene el rectángulo que rodea al pájaro (posición y tamaño).

        self.rect[0] = SCREEN_WIDTH / 6
        # Posición inicial en x: un poco a la izquierda del centro.

        self.rect[1] = SCREEN_HEIGHT / 2
        # Posición inicial en y: al centro vertical de la pantalla.

        self.image.convert_alpha()
        # Optimiza la imagen para que se renderice más rápido con transparencia.


    def update(self):
        self.speed += GRAVITY # Aumenta la velocidad hacia abajo (simula gravedad).
        self.rect[1] += self.speed # Actualiza la posición vertical del pájaro según la velocidad.

    def bump(self):
        self.speed =-SPEED      # Invierte la velocidad para que el pájaro suba. # Se activa cuando el jugador presiona una tecla o hace un gesto.

#clase para las tuberias
class Pipe(pygame.sprite.Sprite):
    def __init__(self, inverted, xpos, ysize):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.Surface((PIPE_WIDTH, PIPE_HEIGHT), pygame.SRCALPHA)
        if inverted:
            self.image.blit(pipe_inverted_img,(0,0)) #si es el de arriba, lo volteamos
        else:
            self.image.blit(pipe_img, (0, 0)) #si es el de abajo, lo dejamos normal
        self.rect = self.image.get_rect()

        self.rect[0] = xpos #posicion horizontal del tubo
        self.passed = False #si ya fue pasado por el pájaro
        if inverted:
            self.rect[1] =-(self.rect[3] -ysize) #posicion vertical para el de arriba
        else:
            self.rect[1] = SCREEN_HEIGHT-ysize  #posicion para el de abajo
    def update(self):
        self.rect[0]-= GAME_SPEED #movemos el tubo hacia la izquierda

#clase para suelo
class Ground(pygame.sprite.Sprite):
    def __init__(self, xpos):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.Surface((SCREEN_WIDTH, GROUND_HEIGHT), pygame.SRCALPHA)
        self.image.blit(grounds, (0,0))
        self.rect = self.image.get_rect()
        self.rect[0] =xpos #posicion horizontal
        self.rect[1] = SCREEN_HEIGHT - GROUND_HEIGHT #posicion vertical fija (abajo de todo)

    def update(self):
        self.rect[0] -= GAME_SPEED #movemos el suelo hacia la izquierda
#rect[0] posicion de X, rect[1] posicion en y, rect[2] ancho y rect[3] para altura
#función para saber si un sprite ya salió de la pantalla
def offScreen(sprite):
    return sprite.rect[0] < -(sprite.rect[2]) #si el sprite se va de la pantalla (a la izquierda) lo eliminamos
#genera un par de tubos (arriba y abajo) con espacio entre ellos
def randomSize(xpos):
    size = random.randint(100,300) #altura aleatoria para el de abajo
    pipe = Pipe(False, xpos, size)
    pipe_inverted =Pipe(True, xpos,SCREEN_HEIGHT - size -PIPE_GAP)#altura del de arriba (restamos el espacio entre los tubos)
    return pipe, pipe_inverted
#clase para el score
class Score(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.score = 0
        self.font = pygame.font.Font(None, 25)
        self.image = pygame.Surface((100, 50), pygame.SRCALPHA)
        self.rect = self.image.get_rect()
        self.rect.topleft = (5, 5)

    def update(self):
        self.text = self.font.render("SCORE: "+str(int(self.score)), 1, WHITE)
        self.image = self.text

class DetectarManos:
    def __init__(self, mode=False, max_hands=1, detection_confidence=0.5, tracking_confidence=0.5):
        self.mode = mode
        self.max_hands = max_hands
        self.detection_confidence = detection_confidence
        self.tracking_confidence = tracking_confidence
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=self.mode,
            max_num_hands=self.max_hands,
            min_detection_confidence=self.detection_confidence,
            min_tracking_confidence=self.tracking_confidence
        )
        self.tip = [4, 8]
        self.lista_landmarks = []

    def detectar_manos(self, frame):
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        resultados = self.hands.process(frame_rgb)
        self.lista_landmarks = []

        if resultados.multi_hand_landmarks:
            for hand_landmarks in resultados.multi_hand_landmarks:
                for id in self.tip:
                    lm = hand_landmarks.landmark[id]
                    h, w, _ = frame.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    self.lista_landmarks.append([id, cx, cy])

    def calcular_distancia(self):
        if len(self.lista_landmarks) < 2:
            return float('inf')
        x1, y1 = self.lista_landmarks[0][1:]
        x2, y2 = self.lista_landmarks[1][1:]
        return math.hypot(x2 - x1, y2 - y1)

def main():
    bird_group = pygame.sprite.Group()
    bird = Bird()
    bird_group.add(bird)
    ground_group =pygame.sprite.Group()
    for i in range(2):
        ground = Ground(SCREEN_WIDTH *i) #dos suelos, uno detras del otro
        ground_group.add(ground)
    pipe_group = pygame.sprite.Group()
    for i in range(2):
        pipes = randomSize(SCREEN_WIDTH *i+800)#generamos tubos
        pipe_group.add(pipes[0])
        pipe_group.add(pipes[1])
    score = Score()
    score_group = pygame.sprite.Group()
    score_group.add(score)
    clock = pygame.time.Clock()
    game_started = False
    cap = cv2.VideoCapture(0) #abrimos la cámara
    detector =DetectarManos(detection_confidence=0.5, tracking_confidence=0.5) #inicializamos el detector
    while True:
        success, frame = cap.read() #leemos frame de la camara
        if not success:
            print("No se pudo acceder a la camara.")
            break
        detector.detectar_manos(frame)
        distance = detector.calcular_distancia()  # medimos distancia entre dedos
        if distance < 30: #gesto de juntar dedos
            if not game_started:
                game_started =True #empezamos el juego
            bird.bump() #el pajaro salta
        #capturacion de eventos del usuario
        for event in pygame.event.get():
            if event.type == QUIT: #si se cierra la ventana se termina el juego
                pygame.quit()
                cap.release()
                cv2.destroyAllWindows()
                sys.exit()
        screen.fill(BLUE)#asignación de color azul
        screen.blit(background, (0, 0)) #dibujamos fondo
        if game_started:
            bird_group.update()
            ground_group.update()
            pipe_group.update()
            score_group.update()
            if offScreen(ground_group.sprites()[0]): #si un suelo se va, lo reemplazamos
                ground_group.remove(ground_group.sprites()[0])
                new_ground = Ground(SCREEN_WIDTH - 20)
                ground_group.add(new_ground)
            if offScreen(pipe_group.sprites()[0]): #si un tubo se va, generamos nuevos
                pipe_group.remove(pipe_group.sprites()[0])
                pipe_group.remove(pipe_group.sprites()[0])
                pipes = randomSize(SCREEN_WIDTH * 2)
                pipe_group.add(pipes[0])
                pipe_group.add(pipes[1])
            for pipe in pipe_group:
                if pipe.rect[0] + PIPE_WIDTH < bird.rect[0] and not pipe.passed:
                    pipe.passed = True
                    score.score +=0.5 #hice la novatada de incrementar 0.5
            if pygame.sprite.groupcollide(bird_group, pipe_group, False, False) or pygame.sprite.groupcollide(bird_group, ground_group, False, False):
                return #si chocamos con algo se termina el juego
        bird_group.draw(screen)
        pipe_group.draw(screen)
        ground_group.draw(screen)
        score_group.draw(screen)
        pygame.display.update()
        clock.tick(60)#puse mas fps pa ver si mejoraba pero alch no vi mucho cambio supongo que es por la camara
    cv2.destroyAllWindows()
    cap.release()
    pygame.quit()
if __name__ == '__main__':
    main()












# Autor: Erik
# Inspirado en el proyecto original de Eliud SG:Creador original